In [5]:
import pandas as pd
pd.set_option('max_columns', None)
import time
import itertools

from nba_api.stats.endpoints import leaguestandings
from nba_api.stats.endpoints import teamyearbyyearstats

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [15]:
standings_all_years = pd.DataFrame()

In [17]:
for year in range (1982, 2023):
    standings = leaguestandings.LeagueStandings(league_id="00", season=year, season_type="Regular Season")
    standingsdf = standings.get_data_frames()[0]
    standings_all_years = standings_all_years.append(standingsdf, ignore_index=True)
    print(year)
    time.sleep(2)
    

1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [20]:
standings_all_years.to_csv('standings_complete.csv', index=False)

In [60]:
standings_all_years = pd.read_csv('standings_complete.csv')

In [61]:
standings_all_years.head(50)

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,LeagueRank,Record,HOME,ROAD,L10,Last10Home,Last10Road,OT,ThreePTSOrLess,TenPTSOrMore,LongHomeStreak,strLongHomeStreak,LongRoadStreak,strLongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,strCurrentHomeStreak,CurrentRoadStreak,strCurrentRoadStreak,CurrentStreak,strCurrentStreak,ConferenceGamesBack,DivisionGamesBack,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,EliminatedConference,EliminatedDivision,AheadAtHalf,BehindAtHalf,TiedAtHalf,AheadAtThird,BehindAtThird,TiedAtThird,Score100PTS,OppScore100PTS,OppOver500,LeadInFGPCT,LeadInReb,FewerTurnovers,PointsPG,OppPointsPG,DiffPointsPG,vsEast,vsAtlantic,vsCentral,vsWest,vsPacific,vsMidwest,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS,vsSoutheast,vsNorthwest,vsSouthwest
0,0,21982,1610612747,Los Angeles,Lakers,West,42-18,1,NaN,Pacific,21-9,1,58,24,0.707,NaN,58-24,33-8,25-16,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0,109.5,5.5,16-6,4-6,12-0,42-18,21-9,21-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38-11,20-13,NaN,NaN,NaN
1,0,21982,1610612755,Philadelphia,76ers,East,44-14,1,NaN,Atlantic,15-9,1,65,17,0.793,NaN,65-17,35-6,30-11,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.1,104.4,7.7,44-14,15-9,29-5,21-3,10-2,11-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-7,22-10,NaN,NaN,NaN
2,0,21982,1610612756,Phoenix,Suns,West,42-18,2,NaN,Pacific,21-9,2,53,29,0.646,NaN,53-29,32-9,21-20,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,102.0,5.0,11-11,3-7,8-4,42-18,21-9,21-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30-22,23-7,NaN,NaN,NaN
3,0,21982,1610612749,Milwaukee,Bucks,East,36-22,2,NaN,Central,22-7,1,51,31,0.622,NaN,51-31,31-10,20-21,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.6,102.2,4.4,36-22,14-15,22-7,15-9,6-6,9-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33-18,18-13,NaN,NaN,NaN
4,0,21982,1610612759,San Antonio,Spurs,West,39-21,2,NaN,Midwest,21-9,1,53,29,0.646,NaN,53-29,31-10,22-19,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.3,110.7,3.6,14-8,5-5,9-3,39-21,18-12,21-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32-21,21-8,NaN,NaN,NaN
5,0,21982,1610612738,Boston,Celtics,East,37-21,3,NaN,Atlantic,14-10,2,56,26,0.683,NaN,56-26,33-8,23-18,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.1,106.7,5.4,37-21,14-10,23-11,19-5,8-4,11-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38-12,18-14,NaN,NaN,NaN
6,0,21982,1610612760,Seattle,SuperSonics,West,36-24,4,NaN,Pacific,14-16,3,48,34,0.585,NaN,48-34,29-12,19-22,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.0,106.8,3.2,12-10,3-7,9-3,36-24,14-16,22-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28-23,20-11,NaN,NaN,NaN
7,0,21982,1610612751,New Jersey,Nets,East,35-23,4,NaN,Atlantic,11-13,3,49,33,0.598,NaN,49-33,30-11,19-22,NaN,NaN,NaN,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.8,103.0,2.8,35-23,11-13,24-10,14-10,8-4,6-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33-18,16-15,NaN,NaN,NaN
8,0,21982,1610612757,Portland,Trail Blazers,West,34-26,5,NaN,Pacific,16-14,4,46,36,0.561,NaN,46-36,31-10

In [62]:
standings_selected = standings_all_years[['SeasonID', 'TeamID', 'TeamName', 'OppPointsPG', 'HOME', 'PlayoffRank']]

In [67]:
standings_selected['SeasonID'] = standings_selected['SeasonID'].astype(str)

<ipython-input-67-b23c11e0b62a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standings_selected['SeasonID'] = standings_selected['SeasonID'].astype(str)


In [68]:
standings_selected['SeasonID'] = standings_selected['SeasonID'].str[1:]

<ipython-input-68-6855e1121c5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standings_selected['SeasonID'] = standings_selected['SeasonID'].str[1:]


In [71]:
standings_selected['SeasonID'] = standings_selected['SeasonID'].astype(int)

<ipython-input-71-3f9a90e8374d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standings_selected['SeasonID'] = standings_selected['SeasonID'].astype(int)


In [73]:
standings_selected = standings_selected[standings_selected['SeasonID']>1983]

In [74]:
standings_selected

,SeasonID,TeamID,TeamName,OppPointsPG,HOME,PlayoffRank
46,1984,1610612738,Celtics,108.1,35-6,1
47,1984,1610612747,Lakers,110.9,36-5,1
48,1984,1610612743,Nuggets,117.6,34-7,2
49,1984,1610612749,Bucks,104.0,36-5,2
50,1984,1610612745,Rockets,109.5,29-12,3
...,...,...,...,...,...,...
1151,2022,1610612757,Trail Blazers,117.4,17-24,13
1152,2022,1610612745,Rockets,118.6,14-27,14
1153,2022,1610612766,Hornets,117.2,13-28,14
1154,2022,1610612759,Spurs,123.1,14-27,15


In [75]:
standings_selected['PlayoffRank'].value_counts()

1     82
4     82
6     81
11    81
2     80
8     78
5     77
9     77
10    76
7     75
3     74
12    71
13    68
14    61
15    47
Name: PlayoffRank, dtype: int64

In [76]:
standings_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 46 to 1155
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SeasonID     1110 non-null   int32  
 1   TeamID       1110 non-null   int64  
 2   TeamName     1110 non-null   object 
 3   OppPointsPG  1110 non-null   float64
 4   HOME         1110 non-null   object 
 5   PlayoffRank  1110 non-null   int64  
dtypes: float64(1), int32(1), int64(2), object(2)
memory usage: 56.4+ KB


In [77]:
standings_selected.to_csv('standings_selected_feat.csv', index=False)

In [91]:
result_all_teams = pd.DataFrame()

In [93]:
for team in pd.unique(standings_selected['TeamID']):

    by_team = teamyearbyyearstats.TeamYearByYearStats(team_id=team, per_mode_simple='PerGame', season_type_all_star="Regular Season") 
    by_team = by_team.get_data_frames()[0]
    by_team['YEAR'] = by_team['YEAR'].str[:4]
    by_team['AST_TOV_RAT'] = by_team['AST']/by_team['TOV']
    by_team = by_team.drop(['AST', 'TOV'], axis=1)
    by_team = by_team[['TEAM_ID', 'TEAM_NAME', 'YEAR', 'WIN_PCT', 'FG_PCT', 'REB', 'AST_TOV_RAT', 'PTS']]
    print(team)
    
    result_all_teams = result_all_teams.append(by_team, ignore_index=True)
    time.sleep(2)

1610612739
1610612744
1610612759
1610612761
1610612748
1610612760
1610612737
1610612746
1610612738
1610612757
1610612742
1610612766
1610612754
1610612763
1610612745
1610612765
1610612741
1610612762
1610612758
1610612764
1610612743
1610612753
1610612740
1610612749
1610612750
1610612752
1610612751
1610612756
1610612747
1610612755


In [98]:
result_all_teams['YEAR'] =result_all_teams['YEAR'].astype(int)

In [121]:
result_all_teams[result_all_teams['FG_PCT']==0].head(50)

,index,TEAM_ID,TEAM_NAME,YEAR,WIN_PCT,FG_PCT,REB,AST_TOV_RAT,PTS
0,3,1610612739,Cavaliers,1973,0.354,0.0,45.9,1.329787,100.3
1,4,1610612739,Cavaliers,1974,0.488,0.0,43.4,1.303371,99.0
2,5,1610612739,Cavaliers,1975,0.598,0.0,46.1,1.388889,101.7
3,6,1610612739,Cavaliers,1976,0.524,0.0,47.3,1.363636,102.1
4,7,1610612739,Cavaliers,1977,0.524,0.0,47.1,1.254438,104.4
5,8,1610612739,Cavaliers,1978,0.366,0.0,42.5,1.303571,106.5
6,9,1610612739,Cavaliers,1979,0.451,0.0,45.0,1.538922,114.1
7,10,1610612739,Cavaliers,1980,0.341,0.0,42.7,1.441176,105.7
8,11,1610612739,Cavaliers,1981,0.183,0.0,41.0,1.416149,103.2
51,81,1610612744,Warriors,1973,0.537,0.0,53.8,1.197044,109.9


In [122]:
result_all_teams = result_all_teams[result_all_teams['YEAR']>1981].reset_index()

In [128]:
result_all_teams = result_all_teams.drop(['level_0', 'index'], axis=1)
result_all_teams

,TEAM_ID,TEAM_NAME,YEAR,WIN_PCT,FG_PCT,REB,AST_TOV_RAT,PTS
0,1610612739,Cavaliers,1982,0.280,0.465,43.7,1.127660,97.1
1,1610612739,Cavaliers,1983,0.341,0.465,43.9,1.450617,102.3
2,1610612739,Cavaliers,1984,0.439,0.471,44.5,1.514793,108.6
3,1610612739,Cavaliers,1985,0.354,0.480,43.2,1.465116,107.8
4,1610612739,Cavaliers,1986,0.378,0.470,44.8,1.182741,104.4
...,...,...,...,...,...,...,...,...
1181,1610612755,76ers,2019,0.589,0.468,45.4,1.816901,110.7
1182,1610612755,76ers,2020,0.681,0.476,45.1,1.645833,113.6
1183,1610612755,76ers,2021,0.622,0.466,42.3,1.896000,109.9
1184,1610612755,76ers,2022,0.659,0.487,40.9,1.839416,115.2


In [129]:
result_all_teams.to_csv('result_all_teams.csv', index=False)

In [33]:
all_rosters = pd.DataFrame()

In [78]:
for team in pd.unique(standings_selected['TeamID']):
    for year in range(1984, 2023):
        roster = commonteamroster.CommonTeamRoster(team_id=team, season=year)
        roster = roster.get_data_frames()[0]
        all_rosters = all_rosters.append(roster, ignore_index=True)
        print(team, year)
        time.sleep(1)

1610612745 1984
1610612745 1985
1610612745 1986
1610612745 1987
1610612745 1988
1610612745 1989
1610612745 1990
1610612745 1991
1610612745 1992
1610612745 1993
1610612745 1994
1610612745 1995
1610612745 1996
1610612745 1997
1610612745 1998
1610612745 1999
1610612745 2000
1610612745 2001
1610612745 2002
1610612745 2003
1610612745 2004
1610612745 2005
1610612745 2006
1610612745 2007
1610612745 2008
1610612745 2009
1610612745 2010
1610612745 2011
1610612745 2012
1610612745 2013
1610612745 2014
1610612745 2015
1610612745 2016
1610612745 2017
1610612745 2018
1610612745 2019
1610612745 2020
1610612745 2021
1610612745 2022
1610612755 1984
1610612755 1985
1610612755 1986
1610612755 1987
1610612755 1988
1610612755 1989
1610612755 1990
1610612755 1991
1610612755 1992
1610612755 1993
1610612755 1994
1610612755 1995
1610612755 1996
1610612755 1997
1610612755 1998
1610612755 1999
1610612755 2000
1610612755 2001
1610612755 2002
1610612755 2003
1610612755 2004
1610612755 2005
1610612755 2006
16106127

1610612746 1990
1610612746 1991
1610612746 1992
1610612746 1993
1610612746 1994
1610612746 1995
1610612746 1996
1610612746 1997
1610612746 1998
1610612746 1999
1610612746 2000
1610612746 2001
1610612746 2002
1610612746 2003
1610612746 2004
1610612746 2005
1610612746 2006
1610612746 2007
1610612746 2008
1610612746 2009
1610612746 2010
1610612746 2011
1610612746 2012
1610612746 2013
1610612746 2014
1610612746 2015
1610612746 2016
1610612746 2017
1610612746 2018
1610612746 2019
1610612746 2020
1610612746 2021
1610612746 2022
1610612758 1984
1610612758 1985
1610612758 1986
1610612758 1987
1610612758 1988
1610612758 1989
1610612758 1990
1610612758 1991
1610612758 1992
1610612758 1993
1610612758 1994
1610612758 1995
1610612758 1996
1610612758 1997
1610612758 1998
1610612758 1999
1610612758 2000
1610612758 2001
1610612758 2002
1610612758 2003
1610612758 2004
1610612758 2005
1610612758 2006
1610612758 2007
1610612758 2008
1610612758 2009
1610612758 2010
1610612758 2011
1610612758 2012
16106127

In [124]:
all_rosters['SEASON'] = all_rosters['SEASON'].astype(int)

In [127]:
all_rosters = all_rosters.rename(columns={'TeamID': 'TEAM_ID', 'SEASON': 'SEASON_ID'})

In [128]:
all_rosters.to_csv('all_rosters_1984.csv', index=False)

In [87]:
player_list = pd.unique(all_rosters['PLAYER_ID']).tolist()

In [95]:
all_players_1984 = pd.DataFrame()

In [105]:
for count, player_id in enumerate(player_list[700:]):
        player = playercareerstats.PlayerCareerStats(player_id=player_id)
        player = player.get_data_frames()[0]
        all_players_1984 = all_players_1984.append(player, ignore_index=True)
        print(count, player_id)
        time.sleep(1)

0 203085
1 203932
2 1630538
3 202326
4 1627854
5 1628432
6 1631128
7 201599
8 202704
9 1631212
10 1631298
11 1628971
12 202397
13 1631221
14 76247
15 187
16 76518
17 76616
18 76670
19 76713
20 76887
21 77034
22 77588
23 77623
24 77626
25 894
26 77879
27 76822
28 77946
29 78030
30 78462
31 76231
32 76252
33 76418
34 76990
35 77419
36 77527
37 78149
38 101
39 78192
40 78481
41 1474
42 77456
43 77632
44 78271
45 1984
46 76268
47 81
48 77057
49 76412
50 77301
51 926
52 783
53 76504
54 1667
55 78247
56 77449
57 138
58 898
59 195
60 785
61 77776
62 77890
63 1432
64 76826
65 80
66 76801
67 211
68 299
69 1425
70 78633
71 404
72 704
73 46
74 386
75 116
76 288
77 457
78 210
79 1036
80 994
81 688
82 1532
83 238
84 906
85 1501
86 423
87 209
88 281
89 1714
90 1726
91 1747
92 219
93 1032
94 330
95 2038
96 679
97 2072
98 21
99 389
100 1563
101 1919
102 2409
103 2436
104 2366
105 1538
106 2429
107 1510
108 2551
109 2091
110 2046
111 1496
112 1730
113 2033
114 2590
115 2585
116 2558
117 1916
118 1983
1

836 202725
837 201858
838 1626199
839 203485
840 1627740
841 1627771
842 1628379
843 204038
844 202779
845 1627770
846 203961
847 203687
848 1627785
849 1629004
850 203564
851 1628383
852 1629648
853 201565
854 1629635
855 203895
856 1628402
857 1628977
858 1630165
859 203585
860 202709
861 1628367
862 1630191
863 1630240
864 1630180
865 1629686
866 1630595
867 1630587
868 1630296
869 1630691
870 1628963
871 1630568
872 1631105
873 1631197
874 1630164
875 1630181
876 1631093
877 1631205
878 202711
879 1630647
880 76249
881 77149
882 77328
883 77541
884 77754
885 77911
886 77952
887 78062
888 78381
889 433
890 78581
891 76368
892 77168
893 77214
894 76085
895 78465
896 76426
897 77054
898 77633
899 78445
900 78464
901 76370
902 77180
903 77361
904 356
905 302
906 76288
907 201
908 76803
909 939
910 78297
911 77362
912 77845
913 78310
914 77358
915 77264
916 716
917 236
918 164
919 212
920 313
921 709
922 78493
923 383
924 954
925 938
926 967
927 710
928 197
929 713
930 1061
931 49
932 2

1575 202810
1576 203158
1577 203461
1578 203816
1579 203467
1580 201567
1581 201619
1582 202682
1583 204002
1584 1627855
1585 1626224
1586 1627790
1587 1628417
1588 1629061
1589 1628395
1590 1629685
1591 1629636
1592 1629760
1593 1629731
1594 1630205
1595 1628422
1596 1630171
1597 1630596
1598 1630612
1599 1630600
1600 76011
1601 76742
1602 77864
1603 77984
1604 78095
1605 76621
1606 76824
1607 78407
1608 930
1609 76831
1610 76472
1611 134
1612 77481
1613 77637
1614 76528
1615 77649
1616 1100
1617 1054
1618 78261
1619 77003
1620 1635
1621 78286
1622 47
1623 474
1624 192
1625 1527
1626 77597
1627 1871
1628 1727
1629 358
1630 1753
1631 2228
1632 2343
1633 448
1634 2486
1635 2418
1636 2560
1637 2047
1638 2469
1639 201199
1640 201169
1641 201981
1642 2735
1643 202081
1644 202220
1645 202371
1646 201632
1647 202091
1648 203933
1649 101122
1650 1626164
1651 203098
1652 1627755
1653 1627733
1654 203901
1655 1628409
1656 1629059
1657 1628994
1658 1629028
1659 1629034
1660 1629665
1661 1629660


2281 1630288
2282 1631132
2283 732
2284 734
2285 733
2286 735
2287 964
2288 1626122
2289 1764
2290 2226
2291 2238
2292 2347
2293 2442
2294 2604
2295 2559
2296 2766
2297 101160
2298 202951
2299 202729
2300 201979
2301 203950
2302 203893
2303 203919
2304 1626150
2305 1627738
2306 1627988
2307 1628397
2308 1628415
2309 1627395
2310 1628991
2311 1629007
2312 1629630
2313 1629634
2314 1629741
2315 1629723
2316 1630214
2317 1630217
2318 1630253
2319 1629681
2320 1630582
2321 1630583
2322 1630533
2323 1631367
2324 1631222
2325 1631246
2326 1631254
2327 1631223
2328 2761
2329 101178
2330 200790
2331 201607
2332 202406
2333 201208
2334 203121
2335 203480
2336 203580
2337 1628429
2338 1629129
2339 1629541
2340 1629627
2341 1627982
2342 1629637
2343 1629597
2344 1630230
2345 1630184
2346 203263
2347 1630529
2348 1630631
2349 1630530
2350 1631220
2351 1630700
2352 1630604


In [112]:
all_players_1984['SEASON_ID'] = all_players_1984['SEASON_ID'].str[:4]

In [115]:
all_players_1984['SEASON_ID'] = all_players_1984['SEASON_ID'].astype(int)

In [129]:
all_players_1984.to_csv('all_players_1984.csv', index=False)

In [121]:
all_players_1984[all_players_1984['SEASON_ID']>1983].tail(60)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
22845,201208,2011,00,1610612740,NOH,27.0,5,0.0,98.0,7,14,0.5,0.0,0.0,0.0,9,18,0.5,8,19,27,3,5,4,8.0,14,23
22846,203121,2012,00,1610612740,NOH,23.0,52,2.0,694.0,44,108,0.407,22.0,56.0,0.393,8,8,1.0,8,70,78,44,17,12,23.0,89,118
22847,203121,2013,00,1610612740,NOP,24.0,45,7.0,723.0,73,166,0.44,25.0,77.0,0.325,29,36,0.806,9,43,52,43,24,9,22.0,88,200
22848,203121,2014,00,1610612740,NOP,25.0,5,1.0,43.0,1,7,0.143,0.0,1.0,0.0,0,0,0.0,0,1,1,2,1,0,1.0,8,2
22849,203121,2017,00,1610612740,NOP,28.0,82,3.0,1944.0,216,487,0.444,147.0,358.0,0.411,58,67,0.866,18,146,164,111,28,17,58.0,160,637
22850,203121,2018,00,1610612740,NOP,29.0,69,15.0,1757.0,189,484,0.39,133.0,364.0,0.365,56,71,0.789,14,114,128,146,40,23,59.0,165,567
22851,203121,2020,00,1610612760,OKC,31.0,18,0.0,197.0,22,48,0.458,17.0,42.0,0.405,12,12,1.0,2,22,24,21,13,5,8.0,13,73
22852,203480,2013,00,1610612766,CHA,24.0,1,0.0,3.0,0,3,0.0,0.0,1.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0,0
22853,203480,2013,00,1610612740,NOP,24.0,3,0.0,27.0,5,12,0.417,3.0,5.0,0.6,1,2,0.5,0,8,8,0,1,2,1.0,3,14
22854,203480,2013,00,0,TOT,24.0,4,0.0,30.0,5,15,0.333,3.0,6.0,0.5,1,2,0.5,0,8,8,0,1,2,1.0,3,14


In [131]:
all_rosters

,TEAM_ID,SEASON_ID,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612738,1984,00,Danny Ainge,Danny,danny-ainge,44,G,6-5,175,"MAR 17, 1959",26.0,3,Brigham Young,76017,None
1,1610612738,1984,00,Larry Bird,Larry,larry-bird,33,F,6-9,220,"DEC 07, 1956",28.0,5,Indiana State,1449,None
2,1610612738,1984,00,Quinn Buckner,Quinn,quinn-buckner,28,G,6-3,190,"AUG 20, 1954",30.0,8,Indiana,76296,None
3,1610612738,1984,00,Rick Carlisle,Rick,rick-carlisle,34,G,6-5,210,"OCT 27, 1959",25.0,R,Virginia,76342,None
4,1610612738,1984,00,M.L. Carr,M.L.,ml-carr,30,F,6-6,205,"JAN 09, 1951",34.0,8,Guilford,76351,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16100,1610612740,2022,00,Jonas Valanciunas,Jonas,jonas-valanciunas,17,C,6-11,265,"MAY 06, 1992",31.0,10,Lietuvos rytas Vilnius,202685,Traded from MEM on 08/07/21
16101,1610612740,2022,00,Larry Nance Jr.,Larry,larry-nance-jr,22,F-C,6-8,245,"JAN 01, 1993",30.0,7,Wyoming,1626204,Traded from POR on 02/08/22
16102,1610612740,2022,00,Trey Murphy III,Trey,trey-murphy-iii,25,F,6-8,206,"JUN 18, 2000",23.0,1,Virginia,1630530,Draft Rights Traded from MEM on 08/07/21
16103,1610612740,2022,00,E.J. Liddell,E.J.,ej-liddell,32,F,6-6,240,"DEC 18, 2000",22.0,R,Ohio State,1630604,#41 Pick in 2022 Draft


In [133]:
rosters_players_combined = all_rosters.merge(all_players_1984, how='inner', on=['PLAYER_ID', 'SEASON_ID', 'TEAM_ID'])

In [137]:
rosters_players_combined.to_csv('rosters_players_combined.csv', index=False)

In [140]:
rosters_players_combined

,TEAM_ID,SEASON_ID,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED,LEAGUE_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612738,1984,00,Danny Ainge,Danny,danny-ainge,44,G,6-5,175,"MAR 17, 1959",26.0,3,Brigham Young,76017,None,00,BOS,26.0,75,73.0,2564.0,419,792,0.529,15.0,56.0,0.268,118,136,0.868,76,192,268,399,122,6,149.0,228,971
1,1610612738,1984,00,Larry Bird,Larry,larry-bird,33,F,6-9,220,"DEC 07, 1956",28.0,5,Indiana State,1449,None,00,BOS,28.0,80,77.0,3161.0,918,1760,0.522,56.0,131.0,0.427,403,457,0.882,164,678,842,531,129,98,248.0,208,2295
2,1610612738,1984,00,Quinn Buckner,Quinn,quinn-buckner,28,G,6-3,190,"AUG 20, 1954",30.0,8,Indiana,76296,None,00,BOS,30.0,75,6.0,858.0,74,193,0.383,0.0,1.0,0.0,32,50,0.64,26,61,87,148,63,2,67.0,142,180
3,1610612738,1984,00,Rick Carlisle,Rick,rick-carlisle,34,G,6-5,210,"OCT 27, 1959",25.0,R,Virginia,76342,None,00,BOS,25.0,38,0.0,179.0,26,67,0.388,0.0,2.0,0.0,15,17,0.882,8,13,21,25,3,0,19.0,21,67
4,1610612738,1984,00,M.L. Carr,M.L.,ml-carr,30,F,6-6,205,"JAN 09, 1951",34.0,8,Guilford,76351,None,00,BOS,34.0,47,0.0,397.0,62,149,0.416,9.0,23.0,0.391,17,17,1.0,21,22,43,24,21,6,24.0,44,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15784,1610612740,2022,00,Jose Alvarado,Jose,jose-alvarado,15,G,6-0,179,"APR 12, 1998",25.0,1,Georgia Tech,1630631,Signed on 08/18/21,00,NOP,25.0,61,10.0,1310.0,201,489,0.411,83.0,247.0,0.336,65,80,0.813,28,113,141,186,67,10,81.0,125,550
15785,1610612740,2022,00,Jonas Valanciunas,Jonas,jonas-valanciunas,17,C,6-11,265,"MAY 06, 1992",31.0,10,Lietuvos rytas Vilnius,202685,Traded from MEM on 08/07/21,00,NOP,31.0,79,79.0,1968.0,446,816,0.547,38.0,109.0,0.349,185,224,0.826,225,579,804,140,20,52,159.0,245,1115
15786,1610612740,2022,00,Larry Nance Jr.,Larry,larry-nance-jr,22,F-C,6-8,245,"JAN 01, 1993",30.0,7,Wyoming,1626204,Traded from POR on 02/08/22,00,NOP,30.0,65,1.0,1381.0,186,305,0.61,14.0,42.0,0.333,55,79,0.696,106,248,354,119,56,37,42.0,133,441
15787,1610612740,2022,00,Trey Murphy III,Trey,trey-murphy-iii,25,F,6-8,206,"JUN 18, 2000",23.0,1,Virginia,1630530,Draft Rights Traded from MEM on 08/07/21,00,NOP,23.0,79,65.0,2448.0,387,800,0.484,202.0,497.0,0.406,172,190,0.905,59,222,281,112,89,43,61.0,155,1148
